In [ ]:
import torch
import non_local_boxes
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
import matplotlib.patches as mpatches

import matplotlib.lines as mlines

# Sugar coating for reloading
%matplotlib inline
%load_ext autoreload
%autoreload 2

# from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('svg')   # in ordert to have unblurred pictures
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

# Definitions

In [ ]:
matrix_to_tensor = non_local_boxes.utils.matrix_to_tensor
tensor_to_matrix = non_local_boxes.utils.tensor_to_matrix

# Boxes as 2x2x2x2 tensors
PR = matrix_to_tensor(non_local_boxes.utils.PR)
PRprime = matrix_to_tensor(non_local_boxes.utils.PRprime)
SR = matrix_to_tensor(non_local_boxes.utils.SR)
I = matrix_to_tensor(non_local_boxes.utils.I)

n = non_local_boxes.evaluate.nb_columns
W_BS09 = non_local_boxes.utils.W_BS09(n)

def WinCHSH(P): return float(non_local_boxes.evaluate.h_flat(P))                    # P is a 2x2x2x2 tensor
def WinCHSHprime(P): return float(non_local_boxes.evaluate.h_prime_flat(P))         # P is a 2x2x2x2 tensor
def BoxProduct(W,P,Q): return non_local_boxes.evaluate.R_tensor(W,P,Q)[:,:,:,:,0]   # P,Q are is a 2x2x2x2 tensor

def Catalan_number(k):
    return int(math.factorial(2*(k-1))/(math.factorial(k)*math.factorial(k-1)))

P = PR
print(WinCHSH(P), WinCHSHprime(P))
for k in range(1,11):
    print(k, Catalan_number(k))

In [ ]:
def Compute_Orbits(P, W, max_depth):
    Orbits = [[P]]  # each all_orbits[k] is a list, which is the orbit of depth k+1
    for k in range(max_depth-1):
          Orbits.append([])
          for l in range(k+1):
                for Q1 in Orbits[l]:
                      for Q2 in Orbits[k-l]:
                            Orbits[k+1].append( BoxProduct(W, Q1, Q2) )
    return Orbits

In [ ]:
def Draw_Orbit(P, P1, P1name, P2, P2name, P3, P3name, W, Wname, max_depth):
    # P, P1, P2, P3 are 2x2x2x2 matrices

    # Draw the initial background triangle:
    x1, y1 = WinCHSHprime(P1), WinCHSH(P1)
    x2, y2 = WinCHSHprime(P2), WinCHSH(P2)
    x3, y3 = WinCHSHprime(P3), WinCHSH(P3)
    triangle = plt.Polygon(np.array([[x1, y1], [x2,y2], [x3, y3]]), color=hsv_to_rgb([0.475, 1, 0.9]))
    plt.gca().add_patch(triangle)

    BBLMTU_value = (3+math.sqrt(6))/6
    BBLMTU_rectangle = plt.Polygon(np.array([[0.5, 1.0], [0.5,BBLMTU_value], [1.5-BBLMTU_value, BBLMTU_value]]), color=hsv_to_rgb([0.475, 1, 0.5]))
    plt.gca().add_patch(BBLMTU_rectangle)

    # Check if our coordinate system is good, i.e. if projected points aren't aligned:
    if (x1 - x3)*(y2-y3)==(x2-x3)*(y1-y3):
            print("WARNING: Error in the coordinate system.")
            return None

    # Draw the orbit:
    Orbits = Compute_Orbits(P, W, max_depth)
    for k in range(len(Orbits)):
        color = hsv_to_rgb([k/(7*(len(Orbits)-1)), 1, 1])
        for Q in Orbits[k]:  # Q is a tensor
            # Compute the abscissa and the ordinate:
            x = WinCHSHprime(Q)
            if x>=0.5:
                y = WinCHSH(Q)
                plt.plot(x, y, 'o', markersize=4, color=color)
    
    plt.text(WinCHSHprime(P), WinCHSH(P), "  ← starting box $\mathbf{P}$", horizontalalignment='left', verticalalignment = 'center')
    plt.text(x1+0.001, y1, P1name, horizontalalignment='center',weight="bold")
    plt.text(x2, y2+0.007, P2name, verticalalignment='center',weight="bold")
    plt.text(x3, y3-0.007, P3name, verticalalignment='center',weight="bold")
    plt.title("Orbit of a box $\mathbf{P}$ until depth $k=$"+str(max_depth)+" (wiring $W=$"+Wname+")")
    plt.xlabel("$\mathbb{P}($winning at CHSH'$)$")
    plt.ylabel("$\mathbb{P}($winning at CHSH$)$")
    handles = []
    handles.append( mpatches.Patch(color=hsv_to_rgb([0.475, 1, 0.5]), label='Collapsing $\mathcal{N\!S}$-boxes from [BBLMTU06].') )
    handles.append( mpatches.Patch(color=hsv_to_rgb([0.475, 1, 0.8]), label='Other $\mathcal{N\!S}$-boxes.') )
    handles.append( mlines.Line2D([], [],color=hsv_to_rgb([0.0, 1, 1]), marker='o',ls='', label='Orbit of depth $k=$'+str(1)) )
    handles.append(  mlines.Line2D([], [],color=hsv_to_rgb([1/14, 1, 1]), marker='o',ls='', label='Orbit of depth $k=$'+str(max_depth//2)) )
    handles.append( mlines.Line2D([], [],color=hsv_to_rgb([1/7, 1, 1]), marker='o',ls='', label='Orbit of depth $k=$'+str(max_depth)) )
    plt.legend(handles=handles)
    plt.show()


# Compute the Orbit

In [ ]:
alpha = 0.47
beta = 0.51
P = alpha*PR + beta*SR + (1-alpha-beta)*I

Orbits = Draw_Orbit(
    P = P,
    P1 = PR,
    P1name = "PR",
    P2 = SR,
    P2name = "SR",
    P3 = (I+PR)/2,
    P3name = "(I+PR)/2",
    W = W_BS09,
    Wname = "$W_{bs}$",
    max_depth = 12
)

In [ ]:
alpha = 0.47
beta = 0.51
P = alpha*PR + beta*SR + (1-alpha-beta)*I

Orbits = Draw_Orbit(
    P = P,
    P1 = PR,
    P1name = "PR",
    P2 = SR,
    P2name = "SR",
    P3 = (I+PR)/2,
    P3name = "(I+PR)/2",
    W = non_local_boxes.utils.W_FWW09(n),
    Wname = "$W_{\oplus}$",
    max_depth = 12
)